# Inport the relevent data

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

mnist=input_data.read_data_sets('MNIST_data/',one_hot=True) #one hot encoding true

C:\Users\Pritam\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
C:\Users\Pritam\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
C:\Users\Pritam\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


# Outline our model

In [16]:
input_size=784
output_size=10
hidden_layer_size=100

#removes all previous stored graphs or data
tf.reset_default_graph()

inputs=tf.placeholder(tf.float32,[None,input_size])
targets=tf.placeholder(tf.float32,[None,output_size])

#for layer 1
weights_1=tf.get_variable('weights_1',[input_size,hidden_layer_size])
biases_1=tf.get_variable('biases_1',[hidden_layer_size])

outputs_1=tf.nn.relu(tf.matmul(inputs,weights_1)+biases_1) #layer one activation function relu

#for layer 2
weights_2=tf.get_variable('weights_2',[hidden_layer_size,hidden_layer_size])
biases_2=tf.get_variable('biases_2',[hidden_layer_size])

outputs_2=tf.nn.relu(tf.matmul(outputs_1,weights_2)+biases_2) #layer two activation function relu

#final layer 
weights_3=tf.get_variable('weights_3',[hidden_layer_size,output_size])
biases_3=tf.get_variable('biases_3',[output_size])

outputs=tf.matmul(outputs_2,weights_3)+biases_3 #output without activation function


#1st logit function then softmax then cross_entropy loss calculation
#numaricaly stable function when we deals with very small nnumbers
cross_entropy_loss=tf.nn.softmax_cross_entropy_with_logits(logits=outputs,labels=targets) #returns list of losses
mean_loss=tf.reduce_mean(cross_entropy_loss) #<<<<<<<<<<

#optimization method  Adam
optimizer=tf.train.AdamOptimizer(learning_rate=0.001).minimize(mean_loss)#back_prob also performed here #<<<<<<<<<

#prediction accuracy
#equal returns 1 or 0 when values matched or not matched respectively
#output_equals_target is a vactor of 0's (missmatch) and 1's (match) 
#argmax returs the index(r,c) of the maximum value for each target and output list, we need only the column number 
output_equals_target=tf.equal(tf.arg_max(outputs,1),tf.arg_max(targets,1))#argmax retruns row &col num here 1 means only cloumn
accuracy=tf.reduce_mean(tf.cast(output_equals_target,tf.float32)) #<<<<< #cast->[0,1,1]=>[0.0,1.0,1.0]

#creating sessions
sess=tf.InteractiveSession()

#initialize all variables 
initializer=tf.global_variables_initializer() #<<<<<<<
sess.run(initializer)

#batching total 100 batches required
batch_size=100 #batches of max 100 entries
number_of_batches=mnist.train._num_examples // batch_size # // returns int in cealing format

max_epochs=15

previous_validation_loss=9999999. #. makes an int to float 

for epoch_counter in range(max_epochs):
    current_epoch_loss=0. #trainig loss 
    total_batch_loss=0.
    
    for batch_counter in range(number_of_batches):
        batch_inputs,batch_targets=mnist.train.next_batch(batch_size) #inputs and target for each batch
        #getting minimized batch loss with backpropagation
        _,current_batch_loss=sess.run([optimizer,mean_loss],feed_dict={inputs:batch_inputs,targets:batch_targets}) 
        total_batch_loss += current_batch_loss
    current_epoch_loss = total_batch_loss/number_of_batches
    
    #validation after each epoch
    validation_inputs,validation_targets=mnist.validation.next_batch(mnist.validation._num_examples)
    validation_loss,validation_accuracy=sess.run([mean_loss,accuracy],feed_dict={inputs:validation_inputs,targets:validation_targets})
    
    print('Epoch '+str(epoch_counter+1)+
          '. Mean loss: '+'{0:.3f}'.format(current_epoch_loss)+
          '. Validation loss: '+'{0:.3f}'.format(validation_loss)+
          '. Validation accuracy: '+'{0:.2f}'.format(validation_accuracy * 100.)+'%')
    if validation_loss>previous_validation_loss:
        break
    previous_validation_loss=validation_loss
print('end of training')

Epoch 1. Mean loss: 0.330. Validation loss: 0.162. Validation accuracy: 95.36%
Epoch 2. Mean loss: 0.134. Validation loss: 0.111. Validation accuracy: 96.62%
Epoch 3. Mean loss: 0.094. Validation loss: 0.100. Validation accuracy: 96.66%
Epoch 4. Mean loss: 0.070. Validation loss: 0.097. Validation accuracy: 97.08%
Epoch 5. Mean loss: 0.056. Validation loss: 0.094. Validation accuracy: 97.56%
Epoch 6. Mean loss: 0.047. Validation loss: 0.086. Validation accuracy: 97.50%
Epoch 7. Mean loss: 0.038. Validation loss: 0.081. Validation accuracy: 97.62%
Epoch 8. Mean loss: 0.031. Validation loss: 0.079. Validation accuracy: 97.76%
Epoch 9. Mean loss: 0.024. Validation loss: 0.089. Validation accuracy: 97.54%
end of training


# Test Score

In [17]:
test_inputs, test_targets = mnist.test.next_batch(mnist.test._num_examples)
test_loss,test_accuracy=sess.run([mean_loss,accuracy],feed_dict={inputs:test_inputs,targets:test_targets})

test_accuracy_percent = test_accuracy * 100.

# Print the test accuracy formatted in percentages
print('Test accuracy: '+'{0:.2f}'.format(test_accuracy_percent)+'%'+' . Mean loss: '+'{0:.3f}'.format(test_loss))

Test accuracy: 97.47% . Mean loss: 0.084
